In [89]:
import nltk
from nltk.tokenize import word_tokenize

## Eliminate Implication

In [90]:
def left_sentence(expression, index):
    # Initialize parentheses counter
    parentheses_count = 0
    
    # Iterate backward from the given index
    while index >= 0:
        if expression[index] == ')':
            # Increment counter for closing parenthesis
            parentheses_count += 1
        elif expression[index] == '(':
            # Decrement counter for opening parenthesis
            parentheses_count -= 1
            # Check if matching parenthesis for current parenthesis is found
            if parentheses_count == -1:
                # If the previous 2 characters are quantifiers, include them in the left sentence
                if index >= 2 and expression[index - 2:index] in ['∀', '∃']:
                    index -= 2
                break
        # If no parentheses are open and the character is a logical connector or the beginning of the expression
        elif parentheses_count == 0 and (index == 0 or expression[index - 1] in ['∧', '∨', '(', '∀', '∃']):
            break
        # Move index backwards
        index -= 1
    return index


def right_sentence(expression, index):
    # Initialize parentheses counter
    parentheses_count = 0
    
    # Iterate forward from the given index
    while index < len(expression):
        if expression[index] == '(':
            # Increment counter for opening parenthesis
            parentheses_count += 1
        elif expression[index] == ')':
            # Decrement counter for closing parenthesis
            parentheses_count -= 1
        # If no parentheses are open and the character is a logical connector
        elif parentheses_count == 0 and expression[index] in ['∧', '∨']:
            # Move index backwards to include the logical connector
            index -= 1
            break
        # Move index forward
        index += 1
    return index


def eliminate_implication(expression):
    index = 0
    # Iterate through the expression
    while index < len(expression):
        if expression[index:index + 2] == '->':
            # Find the left and right sentences around the implication
            left_sentence_index = left_sentence(expression, index - 1)
            right_sentence_index = right_sentence(expression, index + 1)

            # Extract left and right sentences
            left_expression = expression[left_sentence_index:index].strip()
            right_expression = expression[index + 2:right_sentence_index + 1].strip()

            # Transform implication into its equivalent form
            transformed_expression = f"¬{left_expression} ∨ {right_expression}"
            # Replace implication with its equivalent form in the expression
            expression = expression[:left_sentence_index] + transformed_expression + expression[right_sentence_index + 1:]

            # Update index considering the length change due to transformation
            index = left_sentence_index + len(transformed_expression) - (right_sentence_index - left_sentence_index + 1)
        else:
            # Move index forward if no implication is found
            index += 1
    return expression


## Move Inward Negation

In [91]:
def inward_negation(expression):
    # Remove double negations from the expression.
    expression = another_remove_double_negations(expression)
    # Move negations inside quantifiers such as universal (∀) and existential (∃) quantifiers.
    expression = move_negation_inside_quantifiers(expression)
    # Apply De Morgan's laws to distribute negations over logical connectives (∧ and ∨).
    expression = apply_de_morgans(expression)
    # Return the modified expression.
    return expression

# This function removes instances of double negations in the expression.
def another_remove_double_negations(expression):
    return expression.replace("¬(¬", "(")

# This function moves negations inside quantifiers (∀ and ∃) in the expression.
def move_negation_inside_quantifiers(expression):
    result = ""
    i = 0
    while i < len(expression):
        if expression.startswith("¬∀", i) or expression.startswith("¬∃", i):
            # If a negation (∀ or ∃) is encountered, it moves inside the quantifier.
            quantifier = '∃' if expression[i+1] == '∀' else '∀'
            result += quantifier
            i += 2
            var = expression[i]
            result += var + "¬"
            i += 1
        else:
            # Otherwise, keep the character as it is.
            result += expression[i]
            i += 1
    return result

# This function applies De Morgan's laws to distribute negations over logical connectives (∧ and ∨) in the expression.
def apply_de_morgans(expression):
    while '¬(' in expression:
        start_index = expression.find('¬(') + 2
        balance = 1
        i = start_index
        while i < len(expression) and balance != 0:
            if expression[i] == '(':
                balance += 1
            elif expression[i] == ')':
                balance -= 1
            i += 1
        end_index = i
        inner_expression = expression[start_index:end_index - 1]

        replaced = ""
        if '∧' in inner_expression:
            # If the inner expression contains ∧, De Morgan's law for ∧ is applied.
            terms = inner_expression.split('∧')
            replaced_terms = ["¬" + term for term in terms]
            replaced = " ∨ ".join(replaced_terms)
        elif '∨' in inner_expression:
            # If the inner expression contains ∨, De Morgan's law for ∨ is applied.
            terms = inner_expression.split('∨')
            replaced_terms = ["¬" + term for term in terms]
            replaced = " ∧ ".join(replaced_terms)
        else:
            # If the inner expression is a single variable or negated variable, negate it.
            if inner_expression.startswith("¬"):
                replaced = inner_expression[1:]
            else:
                replaced = "¬" + inner_expression

        # Replace the original expression with the modified one.
        expression = expression[:start_index - 2] + replaced + expression[end_index:]

    return expression

## Remove Double Negation

In [92]:
def remove_double_negation(sentence):
    return sentence.replace('¬¬', '')

## Standardize Variables

In [93]:
def standardize(expressions_string):
    # Splitting the input string into individual expressions based on the delimiter
    expressions = expressions_string.split("', '")

    standardized_expressions = []

    # Iterate over each expression
    for expression in expressions:
        # Standardize the expression and append it to the list of standardized expressions
        new_expression = standardize_expression(expression)
        standardized_expressions.append(new_expression)

    # Joining the standardized expressions into a single string with the original delimiter.
    return "', '".join(standardized_expressions)


def standardize_expression(expression):
    # Set to keep track of used variables
    used_variables = set()
    # Dictionary to store variable replacements
    replacement_map = {}
    # Flags to track the first quantifier found and its variable and replacement
    first_quantifier_found = False
    first_quantifier_var = None
    first_quantifier_replacement = None

    # Function to generate a new variable based on the old variable
    def generate_new_variable(old_var):
        # If a replacement for the old variable already exists, return it
        if old_var in replacement_map:
            return replacement_map[old_var]
        # Generate a new variable name
        new_var = chr(ord('a') + len(used_variables))
        # Ensure uniqueness of the new variable name
        while new_var in used_variables:
            new_var = chr(ord(new_var) + 1)
        # Add the new variable to the set of used variables and update the replacement map
        used_variables.add(new_var)
        replacement_map[old_var] = new_var
        return new_var

    # Function to process quantifiers in the expression
    def process_quantifiers(expr):
        i = 0
        # Iterate over the characters in the expression
        while i < len(expr):
            # Check if the character is a quantifier
            if expr[i] in "∀∃":
                quantifier = expr[i]
                var = expr[i+1]
                # Confirm it's a variable
                if var.islower():
                    # Check if it's the first quantifier found
                    nonlocal first_quantifier_found, first_quantifier_var, first_quantifier_replacement
                    if not first_quantifier_found:
                        # Update the flag and store the first quantifier's variable and replacement
                        first_quantifier_found = True
                        first_quantifier_var = var
                        first_quantifier_replacement = var
                    else:
                        # Generate a new variable and replace occurrences of the old variable with it
                        new_var = generate_new_variable(var)
                        expr = expr[:i+1] + expr[i+1:].replace(var, new_var)
                        # Update the index considering the length change due to variable replacement
                        i += len(new_var) - 1
            # Move to the next character
            i += 1
        return expr

    # Call the function to process quantifiers in the expression
    return process_quantifiers(expression) 

### Move Quantifiers to Left

In [94]:
def move_quantifiers(sentence):
    # Initialize variables to store quantified variables and the rest of the formula
    quantified_vars = []  # List to store tuples of quantifier and associated variable
    formula = []  # List to store the remaining formula

    # Iterate through the characters of the sentence
    i = 0
    while i < len(sentence):
        char = sentence[i]
        # Check if the character is a quantifier
        if char in ['∀', '∃']:
            # If quantifier, find the variable associated with it
            j = i + 1
            var = ""
            while j < len(sentence) and sentence[j].isalnum():
                var += sentence[j]
                j += 1
            # Add the quantifier and its associated variable to the list
            quantified_vars.append((char, var))
            i = j
        else:
            # If not quantifier, add it to the formula
            formula.append(char)
            i += 1

    # Construct the new sentence by moving quantifiers to the left
    new_sentence = ""
    for quantifier, var in quantified_vars:
        new_sentence += quantifier + var
    new_sentence += ''.join(formula)

    return new_sentence


## Skolemization

In [95]:
def skolem(expression, universal_variables=None, skolem_functions=None, used_variables=None):
    # Initialize variables if not provided
    if universal_variables is None:
        universal_variables = []
    if skolem_functions is None:
        skolem_functions = {}
    if used_variables is None:
        used_variables = set()  # Keep track of universal variables used in Skolem functions

    # Nested function to handle quantifiers and variable replacements
    def process_quantifiers(expr):
        new_expr = ""
        default_for_skolem = None  # Default value for Skolem functions
        i = 0
        while i < len(expr):
            if expr[i] in ["∀", "∃"]:
                # Handle universal (∀) and existential (∃) quantifiers
                quantifier = expr[i]
                variable = expr[i + 1]
                if quantifier == "∀":
                    # For universal quantifiers, add the variable to the list of universal variables
                    universal_variables.append(variable)
                    if default_for_skolem is None:
                        # Update the default_for_skolem if not already set
                        default_for_skolem = variable
                    # Include ∀ quantifiers and variables in the new formula
                    new_expr += f"{quantifier}{variable} "
                else:  # quantifier == "∃"
                    if universal_variables:
                        # Use the first unused universal variable for the Skolem function, if available
                        for u_variable in universal_variables:
                            if u_variable not in used_variables:
                                skolem_term = f"f({u_variable})"
                                used_variables.add(u_variable)
                                break
                        else:
                            # Use the default value if all universal variables are used
                            skolem_term = f"f({default_for_skolem})"
                    else:
                        # Use a generic constant when no universal variables are in scope
                        skolem_term = "c"  
                    skolem_functions[variable] = skolem_term
                i += 2  # Skip past the quantifier and variable
                continue
            elif expr[i] in skolem_functions:
                # Replace existential variables with their corresponding Skolem terms
                variable = expr[i]
                skolem_term = skolem_functions[variable]
                new_expr += skolem_term
                i += 1  # Adjust the index after replacement
            else:
                # Directly add other characters to the new formula
                new_expr += expr[i]
                i += 1
        return new_expr

    # Call the nested function to process quantifiers and variable replacements in the expression
    return process_quantifiers(expression)

## Eliminate Universal Quantifers

In [96]:
import re
def eliminate_universal_quantifiers(expression):
    return re.sub(r'∀\w+\s', '', expression)

## Convert to CNF

In [97]:
def convert_to_cnf(sentence):
    # Step 1: Eliminate implications
    sentence = eliminate_implication(sentence)
    print("eliminate_implication:",sentence)
    # Step 2: Move negation inward
    sentence = inward_negation(sentence)
    print("move negation:",sentence)
    # Step 3: Remove double negation
    sentence = remove_double_negation(sentence)
    print("remove double negation:",sentence)
    # Step 4: Standardize variable scope
    sentence = standardize(sentence)
    print("standardize:",sentence)
    # Step 5: Move quantifiers
    sentence = move_quantifiers(sentence)
    print("move quan",sentence)
    # Step 6: Skolemization
    sentence = skolem(sentence)
    print("skolomize:",sentence)

    # Step 7: Eliminate universal quantifiers
    sentence = eliminate_universal_quantifiers(sentence)
    print("elemniate universal",sentence)
    # Join clauses back into CNF sentence
    cnf_sentence = sentence

    return cnf_sentence

# Define the test sentence
test_sentence = "∀x∃y(p(x,y))"
print("original" , test_sentence)
# Call the function and print the result
cnf_result = convert_to_cnf(test_sentence)
print("CNF result:", cnf_result)

################################# Test Cases#############################################

#∀x(p(x)->q(x))
#∀x∃y(p(x,y))
#∃x∀y(¬(p(x,y)))
#∃x(p(x))∧(q(b))
# ¬¬(p(a))
# ∀x∀y∀z(f(x,y)->f(y,z))∧(f(x,z))
# ∀x(∃y(p(y,d,b,c))∨q(d,b))∧p(x,y)->r(x)
# ∀x∃y∃z((l(x,y)∧l(y,z))∧(r(z)->p(z))∧(p(z)->r(z)))
# ∀x((¬(p(a))∧(q(a))))
# ∀x∀z∀u∀w((p(x,f(x),z))∨(p(u,w,w)))
# ∀x∀a∀b(¬(p(x,a,b))∨¬(p(b,b,b)))

original ∀x∃y(p(x,y))
eliminate_implication: ∀x∃y(p(x,y))
move negation: ∀x∃y(p(x,y))
remove double negation: ∀x∃y(p(x,y))
standardize: ∀x∃a(p(x,a))
move quan ∀x∃a(p(x,a))
skolomize: ∀x (p(x,f(x)))
elemniate universal (p(x,f(x)))
CNF result: (p(x,f(x)))


In [98]:
def turn_conjunctions_into_clauses(sentence):
    # Split the sentence into clauses at each 'and'
    clauses = sentence.split('∧')
    return clauses

In [99]:
def rename_variables(sentence):
    # Initialize a dictionary to store mappings of original variable names to unique variable names
    variable_map = {}
    counter = 0

    # Ensure the input is a string by joining the clauses with ' ∨ ' delimiter
    sentence = ' ∨ '.join(sentence)

    # Tokenize the sentence into clauses
    clauses = sentence.split(' ∨ ')

    # Iterate through each clause
    for i in range(len(clauses)):
        clause = clauses[i]
        # Tokenize the clause into words
        tokens = clause.split()
        # Iterate through the tokens
        for j in range(len(tokens)):
            token = tokens[j]
            # Check if the token is a variable
            if token.isalpha() and len(token) == 1:
                # If the variable is already mapped, replace it with the mapped variable name
                if token in variable_map:
                    tokens[j] = variable_map[token]
                else:
                    # If not mapped, generate a new unique variable name and map it
                    counter += 1
                    new_variable_name = 'Var' + str(counter)
                    variable_map[token] = new_variable_name
                    # Replace the variable with the new variable name
                    tokens[j] = new_variable_name
        # Update the clause with the renamed variables
        clauses[i] = ' '.join(tokens)

    # Reconstruct the sentence from the modified clauses
    renamed_sentence = ' ∨ '.join(clauses)

    return renamed_sentence


## Resolution 

In [100]:
def resolve(clauses):
    new_clauses = list(clauses)  # Create a copy of the original clauses
    while True:
        new_clause_count = len(new_clauses)
        for i in range(new_clause_count):
            for j in range(i + 1, new_clause_count):
                clause1 = new_clauses[i]
                clause2 = new_clauses[j]
                resolvents = resolve_clause_pair(clause1, clause2)
                if [] in resolvents:
                    return False  # Empty clause indicates inconsistency
                for resolvent in resolvents:
                    if resolvent not in new_clauses:
                        new_clauses.append(resolvent)
        if len(new_clauses) == new_clause_count:
            break  # No new clauses were added in this iteration
    # If no empty clause is derived, the knowledge base is consistent
    return True

def resolve_clause_pair(clause1, clause2):
    resolvents = []
    for literal1 in clause1:
        for literal2 in clause2:
            if literal1 == negate_literal(literal2):
                resolvent = resolve_literals(clause1, clause2, literal1, literal2)
                if resolvent:
                    resolvents.extend(resolvent)
    return resolvents

def negate_literal(literal):
    if literal.startswith('¬'):
        return literal[1:]
    else:
        return '¬' + literal


def resolve_literals(clause1, clause2, literal1, literal2):
    new_clause1 = [lit for lit in clause1 if lit != literal1]
    new_clause2 = [lit for lit in clause2 if lit != literal2]
    resolvent = new_clause1 + new_clause2
    if not resolvent:
        return [[]]  # Empty clause indicates inconsistency
    return [resolvent]

In [101]:
# Define your two sentences
sentence1 = "∀x(p(x)->q(x))"
sentence2 = "(p(f,a))"


# Convert each sentence to CNF
cnf_sentence1 = convert_to_cnf(sentence1)
print("#################################")
cnf_sentence2 = convert_to_cnf(sentence2)

# Separate the conjunctions into clauses for each CNF sentence
clauses_sentence1 = turn_conjunctions_into_clauses(cnf_sentence1)
clauses_sentence2 = turn_conjunctions_into_clauses(cnf_sentence2)

# Combine the clauses from both sentences
all_clauses = clauses_sentence1 + clauses_sentence2

# Perform resolution
is_inconsistent = resolve(all_clauses)

if is_inconsistent:
    print("The knowledge base is inconsistent.")
else:
    print("The knowledge base is consistent.")

eliminate_implication: ∀x(¬p(x) ∨ q(x))
move negation: ∀x(¬p(x) ∨ q(x))
remove double negation: ∀x(¬p(x) ∨ q(x))
standardize: ∀x(¬p(x) ∨ q(x))
move quan ∀x(¬p(x) ∨ q(x))
skolomize: ∀x (¬p(x) ∨ q(x))
elemniate universal (¬p(x) ∨ q(x))
#################################
eliminate_implication: (p(f,a))
move negation: (p(f,a))
remove double negation: (p(f,a))
standardize: (p(f,a))
move quan (p(f,a))
skolomize: (p(f,a))
elemniate universal (p(f,a))
The knowledge base is inconsistent.
